# Holt Winters Exponential Smoothing Model

In [3]:
# Imports
import sys
import os
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import matplotlib.pyplot as plt

# To import the main.py file
sys.path.append('../')
from python_files import main

# Getting all the data
confirmed_global, deaths_global, recovered_global, country_cases = main.collect_data()

In [4]:
pd.options.display.float_format = '{:.5f}'.format

In [5]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [37]:
def predict(country_name):
    # Getting the confirmed cases of that country
    cases = main.get_new_cases(country_name)
    
    # Removing the zero values 
    is_0 = cases['cases'] != 0
    cases = cases[is_0]
    
    # Making the training and test sets
    split_ratio = 0.8
    train_size = int(split_ratio * len(cases))
    train_df, test_df = cases.iloc[:train_size, :], cases.iloc[train_size:, :]
    train_df, test_df = pd.Series(train_df['cases'].values, train_df['date']), pd.Series(test_df['cases'].values, test_df['date'])
    cases = pd.Series(cases['cases'].values, cases['date'])
    
    # Model
    model = Holt(train_df).fit(smoothing_level = 1, smoothing_trend = 1, optimized = False)
    forecast1 = model.forecast(len(test_df))
    
    print(mape(forecast1, test_df))

In [38]:
predict('India')

8.980962727655774


/home/anand/Documents/Computer Science/Projects/covid-19/covid_env/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/home/anand/Documents/Computer Science/Projects/covid-19/covid_env/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(


In [39]:
predict('Russia')

1.5770715663733594


/home/anand/Documents/Computer Science/Projects/covid-19/covid_env/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/home/anand/Documents/Computer Science/Projects/covid-19/covid_env/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(
